In [3]:
import pandas as pd
import numpy as np
import cv2
import mediapipe as mp
import tensorflow as tf

In [4]:
mpHands = mp.solutions.hands
hands = mpHands.Hands(max_num_hands=1, min_detection_confidence=0.7)
draw = mp.solutions.drawing_utils

In [4]:
def capture_datapoints():
    vid = cv2.VideoCapture(0)
    vid.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    vid.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

    label = input("\nEnter the Gesture Name: ")

    datapoints = []
    data_count = 1

    print(f'\nInitiating Capture for {label}')

    while True:
        _, frame = vid.read()

        if frame is None:
            print("Error: Could not read frame from camera.")
            continue  # Skip to the next iteration of the loop

        frame = cv2.flip(frame, 1)
        y,x,_ = frame.shape


        rgb_img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        res = hands.process(rgb_img)

        landmarks = {'label': label}
        if res.multi_hand_landmarks:
            #? Drawing landmarks on the frame
            for handslms in res.multi_hand_landmarks:
                draw.draw_landmarks(frame, handslms, mpHands.HAND_CONNECTIONS)

                lndmarks = handslms.landmark
                for i in range(len(lndmarks)):
                    landmarks[f'x{i}'] = lndmarks[i].x * x
                    landmarks[f'y{i}'] = lndmarks[i].y * y

        cv2.imshow("frame", frame)

        key = cv2.waitKey(0)
        if key == ord("q"):
            break

        elif key == ord("s"):
            datapoints.append(landmarks)
            print(f'{data_count}) {landmarks}')
            data_count += 1

        elif key == ord("n"):
            label = input("\nEnter the Gesture Name: ")
            print(f"\nInitiating Capture for {label}")
            data_count = 1

    vid.release()
    cv2.destroyAllWindows()

    return datapoints

In [5]:
def refine_data(data):
    res = []
    count = 0
    for i in data:
        if len(i) == 43:
            res.append(i)
        else:
            count+=1

    print(f'{count} data points removed')
    return res

In [7]:
data = capture_datapoints()


Initiating Capture for Backward
1) {'label': 'Backward', 'x0': 293.6052894592285, 'y0': 254.53359603881836, 'x1': 246.20262145996094, 'y1': 240.46391487121582, 'x2': 207.48979568481445, 'y2': 203.9714527130127, 'x3': 191.8403434753418, 'y3': 158.26080322265625, 'x4': 207.6555061340332, 'y4': 125.224928855896, 'x5': 219.63001251220703, 'y5': 161.69797897338867, 'x6': 201.87156677246094, 'y6': 130.77165126800537, 'x7': 209.68395233154297, 'y7': 163.42695236206055, 'x8': 219.69659805297852, 'y8': 173.50895404815674, 'x9': 245.47697067260742, 'y9': 157.58337020874023, 'x10': 228.43029022216797, 'y10': 132.98402309417725, 'x11': 238.2878303527832, 'y11': 178.13279628753662, 'x12': 248.1723403930664, 'y12': 182.48485565185547, 'x13': 272.59469985961914, 'y13': 156.70505046844482, 'x14': 256.8588066101074, 'y14': 137.91781425476074, 'x15': 262.63526916503906, 'y15': 181.0792350769043, 'x16': 270.81836700439453, 'y16': 184.38690662384033, 'x17': 302.7293586730957, 'y17': 157.58272647857666, '

In [8]:
data = refine_data(data)

0 data points removed


In [9]:
df = pd.DataFrame(data)
df.head()

,label,x0,y0,x1,y1,x2,y2,x3,y3,x4,...,x16,y16,x17,y17,x18,y18,x19,y19,x20,y20
0,Backward,293.605289,254.533596,246.202621,240.463915,207.489796,203.971453,191.840343,158.260803,207.655506,...,270.818367,184.386907,302.729359,157.582726,283.740063,141.750698,283.747444,172.621565,291.912708,178.750362
1,Backward,277.758656,246.029348,234.871998,236.349506,197.969513,199.295540,188.843460,152.623129,202.908955,...,261.498756,181.247950,292.193775,152.562261,272.778416,135.976753,271.258221,166.127687,280.659370,172.360754
2,Backward,329.809952,239.655375,293.782234,239.442501,255.076027,199.274182,250.919876,147.912998,270.558777,...,316.298714,180.293341,344.264488,156.556191,333.580055,142.234440,331.998596,170.770926,337.514000,175.113101
3,Backward,382.145996,244.530773,345.716095,241.260824,305.123558,200.795259,299.533424,151.381545,329.183426,...,365.799561,175.578632,390.178528,156.288257,386.641655,138.644886,388.619804,164.498806,391.123581,169.105339
4,Backward,392.345047,270.843544,355.965652,262.760954,323.874435,225.447407,319.478817,181.816950,342.551956,...,382.934456,204.336033,410.889702,183.787079,401.993446,164.556341,401.199150,192.607770,404.791489,199.648004


In [10]:
df.shape

(97, 43)

In [ ]:
old_df = pd.read_csv('hand_gestures.csv')
new_df = pd.concat([old_df, df], ignore_index=True)
#new_df.to_csv('hand_gestures.csv', index=False)

In [ ]:
for i in df.columns:
    for j in i:
        

In [12]:
df.to_csv('hand_gestures.csv', index=False)